<a href="https://colab.research.google.com/github/Srimanth1798/NLP_ASSIGNMENT-2/blob/master/1117462_1dconv_reg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [72]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

url_tr="https://raw.githubusercontent.com/cacoderquan/Sentiment-Analysis-on-the-Rotten-Tomatoes-movie-review-dataset/master/train.tsv"
data=pd.read_csv(url_tr,sep='\t')
data.head(20)

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2
5,6,1,of escapades demonstrating the adage that what...,2
6,7,1,of,2
7,8,1,escapades demonstrating the adage that what is...,2
8,9,1,escapades,2
9,10,1,demonstrating the adage that what is good for ...,2


In [0]:
numSentences = data['SentenceId'].max()

In [0]:
fullSentences = []
curSentence = 0
for i in range(data.shape[0]):
  if data['SentenceId'][i]> curSentence:
    fullSentences.append((data['Phrase'][i], data['Sentiment'][i]))
    curSentence = curSentence +1

In [75]:
len(fullSentences)

8544

In [0]:
fullSentDf = pd.DataFrame(fullSentences,
                                columns=['Phrase', 'Sentiment'])

In [6]:
data['Sentiment'].value_counts()

2    79582
3    32927
1    27273
4     9206
0     7072
Name: Sentiment, dtype: int64

In [76]:
fullSentDf['Sentiment'].value_counts()

3    2325
1    2203
2    1659
4    1282
0    1075
Name: Sentiment, dtype: int64

In [77]:
import nltk
import random
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [0]:
documents = []
for i in range(fullSentDf.shape[0]):
  tmpWords = word_tokenize(fullSentDf['Phrase'][i])
  documents.append((tmpWords, fullSentDf['Sentiment'][i]))

In [79]:
random.seed(9001)
random.shuffle(documents)
print(documents[1][0])

['For', 'its', '100', 'minutes', 'running', 'time', ',', 'you', "'ll", 'wait', 'in', 'vain', 'for', 'a', 'movie', 'to', 'happen', '.']


In [80]:
len(documents)

8544

In [81]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer, LancasterStemmer
porter = PorterStemmer()
lancaster=LancasterStemmer()
wordnet_lemmatizer = WordNetLemmatizer()
stopwords_en = stopwords.words("english")
punctuations="?:!.,;'\"-()"

remove_stopwords = True
useStemming = True
useLemma = False
removePuncs = True

for l in range(len(documents)):
  label = documents[l][1]
  tmpReview = []
  for w in documents[l][0]:
    newWord = w
    if remove_stopwords and (w in stopwords_en):
      continue
    if removePuncs and (w in punctuations):
      continue
    if useStemming:
      newWord = lancaster.stem(newWord)
    if useLemma:
      newWord = wordnet_lemmatizer.lemmatize(newWord)
    tmpReview.append(newWord)
  documents[l] = (' '.join(tmpReview), label)
print(documents[2])

('ian holm conqu frant earthy napoleon', 3)


In [0]:
all_data = pd.DataFrame(documents,
                                columns=['text', 'sentiment'])

x_train_raw, x_test_raw, y_train_raw, y_test_raw = train_test_split(all_data['text'], all_data['sentiment'], test_size=0.3)

In [83]:
len(x_train_raw)

5980

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(stop_words="english",
                             ngram_range=(1, 2))
#vectorizer = TfidfVectorizer(stop_words="english",
#                             ngram_range=(1, 1))
#X = vectorizer.fit_transform(all_data["text"])
#Y = all_data['sentiment']
x_train = vectorizer.fit_transform(x_train_raw)
y_train = y_train_raw
x_test = vectorizer.transform(x_test_raw)
y_test = y_test_raw

In [0]:
x_train_np = x_train.toarray()
y_train_np = np.array(y_train)

# Convert the testing data
x_test_np = x_test.toarray()
y_test_np = np.array(y_test)

In [86]:
x_train_np.shape

(5980, 50669)

In [0]:
import torch
torch.manual_seed(2020)

# Import the 1D convolution layer
# Since we’re inputting a 1-dimensional row of data, we can’t use 2D or 3D
from torch.nn import Conv1d

# Import the max pooling layer
from torch.nn import MaxPool1d

# Import the flatten layer
from torch.nn import Flatten

# Import the linear layer
from torch.nn import Linear

# Import the ReLU & Softmax activation function
from torch.nn.functional import relu, softmax, sigmoid

# Import the DataLoader and TensorDataset libraries from PyTorch
# to work with our datasets
from torch.utils.data import DataLoader, TensorDataset

In [0]:
class CnnClassifier(torch.nn.Module):
  # Define the initialization method
  def __init__(self, batch_size, inputs, outputs):

    # Initialize the superclass and store the parameters
    super(CnnClassifier, self).__init__()
    self.batch_size = batch_size
    self.inputs = inputs
    self.outputs = outputs

    # Define the input layer
    # (input channels, output channels, kernel size)
    self.input_layer = Conv1d(inputs, batch_size, 1)

    # Define a max pooling layer
    # (kernel size)
    self.max_pooling_layer = MaxPool1d(1)

    # Define another convolution layer
    self.conv_layer = Conv1d(batch_size, 128, 1)

    # Define a flatten layer
    self.flatten_layer = Flatten()

    # Define a linear layer
    # (inputs, outputs)
    self.linear_layer = Linear(128, 64)

    # Finally, define the output layer
    self.output_layer = Linear(64, outputs)

  # Define a method to feed inputs through the model
  def feed(self, input):
    # Reshape the entry so it can be fed to the input layer
    # Although we’re using 1D convolution, it still expects a 3D array to 
    # process in a 1D fashion
    input = input.reshape((self.batch_size, self.inputs, 1))

    # Get the output of the first layer and run it through the
    # the ReLU activation function
    output = relu(self.input_layer(input))

    # Get the output of the max pooling layer
    output = self.max_pooling_layer(output)

    # Get the output of the second convolution layer and run it
    # through the ReLU activation function
    output = relu(self.conv_layer(output))

    # Get the output of the flatten layer
    output = self.flatten_layer(output)

    # Get the output of the linear layer and run it through the
    # ReLU activation function
    output = self.linear_layer(output)

    # Finally, get the output of the output layer and return it
    output = self.output_layer(output)
    
    #We use softmax for multi-class classification
    output = softmax(output)
   
    return output

In [0]:
from torch.optim import SGD, Adam

# Import the L1Loss (mean absolute error loss) package from pytorch for
# our performance measure
from torch.nn import L1Loss, CrossEntropyLoss


In [90]:
batch_size = 128

# (batch size, X columns, # output neurons)
model = CnnClassifier(batch_size, x_train.shape[1], 5)

# Set the model to use the GPU for processing
model.cuda()

CnnClassifier(
  (input_layer): Conv1d(50669, 128, kernel_size=(1,), stride=(1,))
  (max_pooling_layer): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
  (conv_layer): Conv1d(128, 128, kernel_size=(1,), stride=(1,))
  (flatten_layer): Flatten()
  (linear_layer): Linear(in_features=128, out_features=64, bias=True)
  (output_layer): Linear(in_features=64, out_features=5, bias=True)
)

In [0]:
def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
    correct = max_preds.squeeze(1).eq(y)
    return (correct.sum() / torch.FloatTensor([y.shape[0]])) / torch.FloatTensor([y.shape[0]])

In [92]:
from sklearn.preprocessing import LabelBinarizer
indicies=range(5)
encoder=LabelBinarizer()
labels=encoder.fit_transform(indicies)
print(labels[1])

[0 1 0 0 0]


In [0]:
def model_loss(model, dataset, train = False, optimizer = None):
  # Cycle through the batches and get the average L1 loss
  performance = L1Loss()
  criterion = CrossEntropyLoss()
  avg_loss = 0
  avg_accu = 0
  count = 0
  
  for input, output in iter(dataset):
    # Get the model's predictions for the training dataset
    predictions = model.feed(input)
    
    # Get the model's loss
    loss = performance(predictions, output)
    
    tmp_accu = categorical_accuracy(predictions, output)
    
    if(train):
      # Clear any errors so they don't cummulate
      optimizer.zero_grad()

      # Compute the gradients for our optimizer
      loss.backward()

      # Use the optimizer to update the model's parameters based on the
      # gradients
      optimizer.step()

    # Store the loss and update the counter
    avg_loss += loss.item()

    # Accumulate performance metrices
    avg_accu += tmp_accu.item()
    count += 1
    
  return avg_loss / count, avg_accu / count

In [0]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [94]:
epochs = 25


optimizer = SGD(model.parameters(), lr=1e-5)
#optimizer = Adam(model.parameters())

# Convert the training set into torch variables for our model using the GPU
# as floats. The reshape is to remove a warning pytorch outputs otherwise.
inputs = torch.from_numpy(x_train_np).cuda().float()
outputs = torch.from_numpy(y_train_np.reshape(y_train_np.shape[0], 1)).cuda().float()

# Create a DataLoader instance to work with our batches
tensor = TensorDataset(inputs, outputs)
loader = DataLoader(tensor, batch_size, shuffle=True, drop_last=True)

# Start the training loop
for epoch in range(epochs):
  # Cycle through the batches and get the average loss
  avg_loss, avg_accu = model_loss(model, loader, train=True, optimizer=optimizer)
    
  print("Epoch " + str(epoch + 1) + ":\n\tLoss = " + str(avg_loss))
  print("Accuracy = " + str(avg_accu))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:61: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:88: UserWarning: Using a target size (torch.Size([128, 1])) that is different to the input size (torch.Size([128, 5])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


Epoch 1:
	Loss = 1.9036005631737087
Accuracy = 0.25883152173913043
Epoch 2:
	Loss = 1.8954483918521716
Accuracy = 0.261039402173913
Epoch 3:
	Loss = 1.8992187769516655
Accuracy = 0.25985054347826086
Epoch 4:
	Loss = 1.9004755771678428
Accuracy = 0.26069972826086957
Epoch 5:
	Loss = 1.8991168711496436
Accuracy = 0.2593410326086957
Epoch 6:
	Loss = 1.9009171465168828
Accuracy = 0.2593410326086957
Epoch 7:
	Loss = 1.900339707084324
Accuracy = 0.2600203804347826
Epoch 8:
	Loss = 1.9008152433063672
Accuracy = 0.2603600543478261
Epoch 9:
	Loss = 1.9022758499435757
Accuracy = 0.25985054347826086
Epoch 10:
	Loss = 1.9042799109997957
Accuracy = 0.2591711956521739
Epoch 11:
	Loss = 1.8984035668165788
Accuracy = 0.2593410326086957
Epoch 12:
	Loss = 1.8992187769516655
Accuracy = 0.2591711956521739
Epoch 13:
	Loss = 1.9016984053280042
Accuracy = 0.258491847826087
Epoch 14:
	Loss = 1.9006454141243645
Accuracy = 0.25985054347826086
Epoch 15:
	Loss = 1.89585600728574
Accuracy = 0.2603600543478261
Epoc

In [105]:
torch.save(model, '/content/drive/My Drive/1117462_1dconv_reg.ipynb')

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type CnnClassifier. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [106]:
model = torch.load('/content/drive/My Drive/1117462_1dconv_reg.ipynb')
model.eval()

CnnClassifier(
  (input_layer): Conv1d(50669, 128, kernel_size=(1,), stride=(1,))
  (max_pooling_layer): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
  (conv_layer): Conv1d(128, 128, kernel_size=(1,), stride=(1,))
  (flatten_layer): Flatten()
  (linear_layer): Linear(in_features=128, out_features=64, bias=True)
  (output_layer): Linear(in_features=64, out_features=5, bias=True)
)

In [97]:
inputs = torch.from_numpy(x_test_np).cuda().float()
outputs = torch.from_numpy(y_test_np.reshape(y_test_np.shape[0], 1)).cuda().float()

# Create a DataLoader instance to work with our batches
tensor = TensorDataset(inputs, outputs)
loader = DataLoader(tensor, batch_size, shuffle=True, drop_last=True)
avg_loss, avg_accu = model_loss(model, loader)
print("The model's L1 loss is: " + str(avg_loss))
print("The model's Accuracy is: " + str(avg_accu))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:61: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:88: UserWarning: Using a target size (torch.Size([128, 1])) that is different to the input size (torch.Size([128, 5])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


The model's L1 loss is: 1.942343783378601
The model's Accuracy is: 0.254296875


In [98]:
from torch.optim import SGD
from torch.nn import L1Loss
!pip install pytorch-ignite
from ignite.metrics import Accuracy, Recall, Precision

In [99]:
batch_size = 128
# (batch size, X columns, Y columns)
model = CnnClassifier(batch_size, x_train.shape[1], 5)
# Set the model to use the GPU for processing
model.cuda()

CnnClassifier(
  (input_layer): Conv1d(50669, 128, kernel_size=(1,), stride=(1,))
  (max_pooling_layer): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
  (conv_layer): Conv1d(128, 128, kernel_size=(1,), stride=(1,))
  (flatten_layer): Flatten()
  (linear_layer): Linear(in_features=128, out_features=64, bias=True)
  (output_layer): Linear(in_features=64, out_features=5, bias=True)
)

In [0]:
def model_loss(model, dataset, train = False, optimizer = None):
  # Cycle through the batches and get the average L1 loss
  performance = L1Loss()
  accu = Accuracy()
  prec = Precision()
  rec = Recall()
  avg_accu = 0;
  avg_rec = 0
  avg_prec = 0
  avg_loss = 0
  count = 0
  for input, output in iter(dataset):
    # Get the model's predictions for the training dataset
    predictions, predictions_ = model.feed(input)
    # Get the model's loss using the float returned variable
    loss = performance(predictions, output)


In [104]:
    accu.update([predictions_, output])
    tmp_accu = accu.compute()
    prec.update([predictions_, output])
    tmp_prec = prec.compute()
    rec.update([predictions_, output])
    tmp_rec = rec.compute()
    if(train):
    # Clear any errors so they don't cummulate
      optimizer.zero_grad()
    # Compute the gradients for our optimizer
      loss.backward()
    # Use the optimizer to update the model's parameters based on the gradients
      optimizer.step()
    # Store the loss and update the counter
    avg_loss += loss.item()
    # Accumulate performance metrices
    avg_accu += tmp_accu
    avg_prec += tmp_prec.item()
    avg_rec += tmp_rec.item()
    count += 1

  return avg_loss / count, avg_accu / count, avg_prec / count, avg_rec / count

IndentationError: ignored